In [1]:
!kaggle competitions download -c sf-crime

 90%|██████████████████████████████████▍   | 37.0M/40.9M [00:00<00:00, 51.9MB/s]
100%|██████████████████████████████████████| 40.9M/40.9M [00:00<00:00, 46.7MB/s]


In [2]:
!unzip -U sf-crime.zip

Archive:  sf-crime.zip
  inflating: sampleSubmission.csv.zip  
  inflating: test.csv.zip            
  inflating: train.csv.zip           


In [3]:
!unzip -U test.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                


In [5]:
!unzip -U train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import random
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd

In [8]:
trainDf = pd.read_csv('train.csv')
trainDf.sample(10)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
591977,2006-12-25 23:12:00,RECOVERED VEHICLE,RECOVERED VEHICLE - STOLEN OUTSIDE SF,Monday,NORTHERN,NONE,GOUGH ST / FELL ST,-122.422791,37.775951
594686,2006-12-03 23:25:00,ASSAULT,BATTERY OF A POLICE OFFICER,Sunday,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411071,37.781751
761595,2004-07-22 15:00:00,LARCENY/THEFT,GRAND THEFT FROM A BUILDING,Thursday,CENTRAL,"ARREST, BOOKED",200 Block of SUTTER ST,-122.404828,37.789760
800968,2004-01-19 20:31:00,DRUG/NARCOTIC,POSSESSION OF COCAINE,Monday,MISSION,"ARREST, BOOKED",MISSION ST / 16TH ST,-122.419672,37.765050
637109,2006-05-02 05:00:00,NON-CRIMINAL,LOST PROPERTY,Tuesday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421
594793,2006-12-03 18:00:00,WARRANTS,WARRANT ARREST,Sunday,MISSION,"ARREST, BOOKED",400 Block of VALENCIA ST,-122.422073,37.766126
111973,2013-11-12 14:10:00,DRUG/NARCOTIC,POSSESSION OF BASE/ROCK COCAINE,Tuesday,NORTHERN,"ARREST, BOOKED",300 Block of HAYES ST,-122.422159,37.776931
113820,2013-11-01 02:13:00,OTHER OFFENSES,TRAFFIC VIOLATION,Friday,INGLESIDE,"ARREST, CITED",MISSION ST / ITALY AV,-122.439501,37.718467
237770,2012-03-06 16:00:00,VANDALISM,"MALICIOUS MISCHIEF, GRAFFITI",Tuesday,PARK,NONE,0 Block of SUNVIEW DR,-122.446515,37.747314
444440,2009-02-07 19:21:00,ASSAULT,BATTERY,Saturday,CENTRAL,UNFOUNDED,0 Block of GEARY ST,-122.404278,37.787919


In [9]:
testDf = pd.read_csv('test.csv')
testDf.sample(10)

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
755847,755847,2004-09-11 09:35:00,Saturday,TARAVAL,2300 Block of 30TH AV,-122.487471,37.743450
755693,755693,2004-09-11 19:00:00,Saturday,NORTHERN,GEARY ST / VAN NESS AV,-122.421396,37.785684
356404,356404,2010-05-25 09:00:00,Tuesday,RICHMOND,4300 Block of BALBOA ST,-122.505365,37.775402
108185,108185,2013-12-06 18:00:00,Friday,SOUTHERN,1ST ST / HOWARD ST,-122.396039,37.788537
621899,621899,2006-08-04 15:00:00,Friday,PARK,500 Block of CENTRAL AV,-122.444432,37.774635
237330,237330,2012-03-12 08:46:00,Monday,BAYVIEW,2400 Block of SAN BRUNO AV,-122.404952,37.730738
359662,359662,2010-05-02 14:01:00,Sunday,PARK,HAIGHT ST / COLE ST,-122.450710,37.769526
633401,633401,2006-06-06 20:25:00,Tuesday,INGLESIDE,MISSION ST / EXCELSIOR AV,-122.433623,37.726236
397679,397679,2009-10-19 11:05:00,Monday,NORTHERN,VANNESS AV / TURK ST,-122.420642,37.781961
717270,717270,2005-03-20 06:00:00,Sunday,CENTRAL,800 Block of SUTTER ST,-122.414390,37.788545


# Remove Outliers from X and Y

## Remove from trainDf

In [ ]:
# X in trainingDf has outliers that equal to the max value.
trainDf['X'].describe()

In [ ]:
quantile_low_X = trainDf['X'].quantile(0.01)
quantile_high_X = trainDf['X'].quantile(0.99)
trainDf = trainDf[(trainDf['X'] < quantile_high_X) & (trainDf['X'] > quantile_low_X)]

In [ ]:
trainDf['X'].describe()

In [ ]:
trainDf['Y'].describe()

In [ ]:
quantile_low_Y = trainDf['Y'].quantile(0.01)
quantile_high_Y = trainDf['Y'].quantile(0.99)
trainDf_filtered = trainDf[(trainDf['Y'] < quantile_high_Y) & (trainDf['Y'] > quantile_low_Y)]

In [ ]:
trainDf['Y'].describe()

## Remove from testDf

In [ ]:
# X in testingDf has outliers that equal to the max value.
testDf['X'].describe()

In [ ]:
quantile_low_X = testDf['X'].quantile(0.01)
quantile_high_X = testDf['X'].quantile(0.99)
testDf = testDf[(testDf['X'] < quantile_high_X) & (testDf['X'] > quantile_low_X)]
testDf['X'].describe()

In [ ]:
testDf['Y'].describe()

# Transform the Dates Feature

In [ ]:
# No column contains null values.
# The data type of the Dates column is now object.
trainDf.info()

In [ ]:
# No column contains null values.
# The data type of the Dates column is now object.
testDf.info()

In [ ]:
# Transforms the data type of the Dates columns to datetime.
trainDf["Dates"] = pd.to_datetime(trainDf["Dates"], format="%Y-%m-%d %H:%M:%S")
testDf["Dates"] = pd.to_datetime(testDf["Dates"], format="%Y-%m-%d %H:%M:%S")

In [ ]:
trainDf.info()

In [ ]:
testDf.info()

In [ ]:
# Extracts the year out of the datetime value.
trainDf["Year"] = trainDf["Dates"].map(lambda x: x.year)
testDf["Year"] = testDf["Dates"].map(lambda x: x.year)

In [ ]:
# Extracts the month out of the datetime value.
trainDf["Month"] = trainDf["Dates"].map(lambda x: x.month)
testDf["Month"] = testDf["Dates"].map(lambda x: x.month)

In [ ]:
# Extracts the day out of the datetime value.
trainDf["Day"] = trainDf["Dates"].map(lambda x: x.day)
testDf["Day"] = testDf["Dates"].map(lambda x: x.day)

In [ ]:
# Extracts the hour out of the datetime value.
trainDf["Hour"] = trainDf["Dates"].map(lambda x: x.hour)
testDf["Hour"] = testDf["Dates"].map(lambda x: x.hour)

In [ ]:
# Extracts the minute out of the datetime value.
trainDf["Minute"] = trainDf["Dates"].map(lambda x: x.minute)
testDf["Minute"] = testDf["Dates"].map(lambda x: x.minute)

In [ ]:
trainDf.sample(10)

In [ ]:
testDf.sample(10)

# Drop Features

In [ ]:
# The Dates is decomposed to multiple columns, so it can be dropped.
trainDf.drop(['Dates'], axis=1, inplace=True)
testDf.drop(['Dates'], axis=1, inplace=True)

# The Descript and Resolution columns do not exist in the testingDf, so they are dropped.
trainDf.drop(['Descript'], axis=1, inplace=True)

trainDf.drop(['Resolution'], axis=1, inplace=True)

In [ ]:
trainDf.sample(10)

In [ ]:
testDf.sample(10)

# Feature Encoding

In [ ]:
def oneHotBind(df, features):
    dummies = pd.get_dummies(df[features])
    res = pd.concat([df, dummies], axis=1)
    res = res.drop(features, axis=1)
    
    return res